## 1. High-Level Overview of the Transformer



The attention mechanism can be summarized by:

$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$

Where \( Q \), \( K \), \( V \) represent queries, keys, and values, each shaped (for each head) as:
$ Q, K, V \in \mathbb{R}^{(\text{seq\_len}) \times (d_k)} $

and $( d_k = \frac{d_\text{model}}{n_\text{heads}} )$.

- **Multi-head** means we split the embedding dimension into $( n_\text{heads} )$ parts, compute the above attention in parallel, then concatenate the outputs.

#### Why Multi-Head?
Instead of relying on a single attention distribution, multiple heads let the model attend to different positions (and different representation subspaces) at each layer.

### Position-wise Feed-Forward Networks (FFN)

After the self-attention sub-layer, each token is passed through a two-layer MLP:
$$
\text{FFN}(\mathbf{x}) = \max(0,\, \mathbf{x} W_1 + b_1)\, W_2 + b_2
$$
where:
- $( W_1 \in \mathbb{R}^{d_\text{model} \times d_\text{ff}} ), ( b_1 \in \mathbb{R}^{d_\text{ff}} )$
- $( W_2 \in \mathbb{R}^{d_\text{ff} \times d_\text{model}} ), ( b_2 \in \mathbb{R}^{d_\text{model}} \$


###Positional Encoding

Since the Transformer is permutation-invariant (due to attention alone), we need to inject sequence-order information into the embeddings.

- **Sinusoidal positional encodings**:
  $$
  PE_{(pos, 2i)} = \sin\Bigl(\frac{pos}{10000^{\frac{2i}{d_\text{model}}}}\Bigr),
  \quad
  PE_{(pos, 2i+1)} = \cos\Bigl(\frac{pos}{10000^{\frac{2i}{d_\text{model}}}}\Bigr)
  $$
- **Learned positional embeddings**: A trainable embedding table for positions.
- **Relative positional encoding**: Focuses on the distance between positions.
- **Rotary embeddings** (RoPE): Introduced in [RoFormer](https://arxiv.org/abs/2104.09864), helpful for extending context length.
- **Hybrid**: Combining sinusoidal or rotary with learned embeddings.

---

### Residual Connections & Layer Normalization

Each sub-layer (Self-Attention or FFN) is wrapped with:
```
x -> SubLayer(x) -> x + SubLayer(x) -> LayerNorm
```
This helps stabilize training and reduce gradient vanishing/exploding in deep networks.




## Coding Transformer

We start wth  three training weight matrices $W_q$, $W_k$, and $W_v$ which will be used to project the embedded input tokens, $x^{(i)}$, into query, key, and value vectors via matrix multiplication:

  - Query vector: $q^{(i)} = W_q \,x^{(i)}$
  - Key vector: $k^{(i)} = W_k \,x^{(i)}$
  - Value vector: $v^{(i)} = W_v \,x^{(i)}$

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.nn import functional as F


In [ ]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)

In [ ]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2


- d_in: dim of the i/p features for each token.
- d_out: dim of the  o/p features for each token.
- context_length: Max number of tokens in the input sequence
- dropout: Dropout probability for regularization in the attention weights.
- qkv_bias:
- Three separate learnable linear transformations are defined to project the input embeddings into  Q,K, and V
- Dropout is applied to the attention weights to prevent overfitting.
- Causal mask,  upper triangular mask with ones above the diagonal and zeros below.



In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

### Stacking multiple single-head attention layers
 We simply stack multiple single-head attention modules to obtain a multi-head attention module: The main idea behind multi-head attention is to run the attention mechanism multiple times (in parallel) with different, learned linear projections. This allows the model to jointly attend to information from different representation subspaces at different positions.



In [ ]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

## Using PyTorch’s `nn.Transformer` Module

---

In [ ]:
import torch.nn.functional as F

transformer_model = nn.Transformer(
    d_model=512,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048
)

# Example shapes:
src = torch.randn(10, 32, 512)  # (src_seq_len, batch_size, d_model)
tgt = torch.randn(20, 32, 512)  # (tgt_seq_len, batch_size, d_model)

out = transformer_model(src, tgt)  # => (tgt_seq_len, batch_size, d_model)

> **Note on Shapes**: PyTorch’s built-in `nn.Transformer` expects input shape `(sequence_length, batch_size, d_model)`

Notes
1. **Attention Mechanisms**:
   - Understand *scaled dot-product* in detail.
   - Distinguish self-attention from cross-attention (encoder-decoder attention).

2. **Positional Encoding Innovations**:
   - Sinusoidal vs. learned vs. relative vs. rotary.
   - The trade-offs in practice (e.g., learned embeddings can be more flexible, but sinusoidal is simpler, rotary better for extrapolation, etc.).

3. **Layer Normalization and Residuals**:
   - Vital for stable deep learning, especially in large models.

4. **Model Scaling**:
   - Modern Transformers can have dozens of layers and thousands of hidden dimensions.
   - Expert knowledge requires understanding of memory and computational constraints, as well as *mixed precision* training, distributed training, etc.

5. **Masking**:
   - **Padding Mask**: Avoid attending to `<pad>` tokens.
   - **Causal Mask**: Prevent the decoder from attending to future tokens.
   - **Cross-Attention** Mask: If needed for ignoring padded portions of the encoder output.

6. **Implementation Details**:
   - Efficiency: GPU memory usage, chunked attention, flash attention, etc.
   - GPU/TPU/HPU performance tuning: half-precision, gradient checkpointing for large models.
   - Large-scale training frameworks (e.g., [DeepSpeed](https://github.com/microsoft/DeepSpeed), [Megatron-LM](https://github.com/NVIDIA/Megatron-LM)).

Additional Notes
1. **Original Paper**: [*Attention Is All You Need*](https://arxiv.org/abs/1706.03762) by Vaswani et al.
2. **Hugging Face Transformers**: For pre-trained models (BERT, GPT, T5, etc.), check out [Hugging Face’s Transformers library](https://github.com/huggingface/transformers).
3. **Performance Optimizations**:
   - *Flash Attention*: [Paper](https://arxiv.org/abs/2205.14135), [Open-source implementation](https://github.com/HazyResearch/flash-attention).
   - *Zero Redundancy Optimizer (ZeRO)* in [DeepSpeed](https://www.deepspeed.ai/).

4. **Advanced Topics**:
   - Techniques like *Adapter Layers* for efficient fine-tuning.
   - *Prompt tuning* or *prefix tuning*.
   - *Sparse Attention* for long sequences, e.g., Longformer, Big Bird.
   - *Retrieval-Augmented Transformers* for knowledge grounding.

---
